In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import os
import re
from typing import List, Tuple, Dict, Union
from typing import Iterable as Iter
from typing import Callable
from sklearn.base import BaseEstimator, TransformerMixin

In [18]:
data = pd.read_csv('../acts_labeled/contratos_idato.csv')
data

,id_ato,id_dodf,tipo_rel,id_rel,anotador_rel,timestamp_rel,tipo_ent,id_ent,anotador_ent,timestamp_ent,offset,length,texto
0,22_20.12.2019.1-R5,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,numero_termo_aditivo,275,matheus_santos,2022-03-04T15:34:41Z,372969,20,QUARTO TERMO ADITIVO
1,22_20.12.2019.1-R5,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,numero_contrato,276,matheus_santos,2022-03-04T15:34:41Z,373005,8,002/2018
2,22_20.12.2019.1-R5,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,orgao_contratante,277,matheus_santos,2022-03-04T15:34:41Z,373098,8,DF/RA II
3,22_20.12.2019.1-R5,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,objeto_aditamento_contratual,278,matheus_santos,2022-03-04T15:34:41Z,373238,481,O presente Termo Aditivo objetiva a alteracao ...
4,22_20.12.2019.1-R5,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,processo_gdf,434,matheus_santos,2022-04-02T22:45:46Z,373072,16,0131-000016/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6691,8_13.8.2019-R5,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,vigencia_contrato,6,alexandre_santana,2022-02-22T02:34:10Z,478144,17,04 (quatro) meses
6692,8_13.8.2019-R5,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,valor_contrato,5,alexandre_santana,2022-02-22T02:34:10Z,478173,9,"76.500,00"
6693,8_13.8.2019-R5,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,data_assinatura_contrato,149,alexandre_santana,2022-03-08T12:03:37Z,478086,10,07/08/2019
6694,8_13.8.2019-R5,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,objeto_contrato,150,alexandre_santana,2022-03-08T12:03:37Z,478106,26,aquisicao de fita eletrica


In [17]:
data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['id_ato'] == '22_20.12.2019.1-R20')]

,id_ato,id_dodf,tipo_rel,id_rel,anotador_rel,timestamp_rel,tipo_ent,id_ent,anotador_ent,timestamp_ent,offset,length,texto
12,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,modalidade_licitacao,4,carla_araujo,2022-02-22T20:46:12Z,376020,17,PREGAO ELETRONICO
13,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,numero_licitacao,5,carla_araujo,2022-02-22T20:46:12Z,376041,8,001/2020
14,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,processo_gdf,6,carla_araujo,2022-02-22T20:46:12Z,376057,6,974002
15,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,orgao_licitante,418,carla_araujo,2022-03-23T15:07:28Z,376094,36,Instituto Brasilia Ambiental (IBRAM)
16,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,sistema_compras,419,carla_araujo,2022-03-23T15:07:28Z,376327,10,Comprasnet
17,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,valor_estimado_contratacao,41,carla_araujo,2022-02-22T21:32:47Z,376551,10,"170.704,26"
18,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,data_abertura_licitacao,11,carla_araujo,2022-02-22T20:46:12Z,376726,10,08/01/2020
19,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,processo_gdf,12,carla_araujo,2022-02-22T20:46:12Z,376767,22,00040-00017799/2019-42
20,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,nome_responsavel,14,carla_araujo,2022-02-22T20:46:12Z,376958,27,KARLA REGINA DA SILVA ROCHA
21,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,R20,carla_araujo,2022-04-04T01:17:11Z,tipo_objeto,467,carla_araujo,2022-04-04T01:16:06Z,376393,9,aquisicao


In [19]:
df = data[['id_ato', 'id_dodf', 'tipo_rel', 'tipo_ent', 'texto']].loc[(data['tipo_ent'] == 'EXTRATO_ADITAMENTO_CONTRATUAL') |\
                                                               (data['tipo_ent'] == 'AVISO_LICITACAO') |\
                                                               (data['tipo_ent'] == 'AVISO_SUSPENSAO_LICITACAO') |\
                                                               (data['tipo_ent'] == 'EXTRATO_CONTRATO') |\
                                                               (data['tipo_ent'] == 'EXTRATO_CONVENIO') |\
                                                               (data['tipo_ent'] == 'AVISO_ANUL_REV_LICITACAO')]

In [20]:
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'EXTRATO_ADITAMENTO_CONTRATUAL')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'AVISO_LICITACAO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'AVISO_SUSPENSAO_LICITACAO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'EXTRATO_CONTRATO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'EXTRATO_CONVENIO')]
data = data[['id_ato', 'id_dodf', 'tipo_rel', 'id_rel', 'anotador_rel',
       'timestamp_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'timestamp_ent',
       'offset', 'length', 'texto']].loc[(data['tipo_ent'] != 'AVISO_ANUL_REV_LICITACAO')]

In [23]:
data = data.reset_index(drop=True)
df = df.reset_index(drop=True)

In [24]:
tokens = data['tipo_ent'].unique()

for token in tokens:
    df[token] = np.nan
        

In [25]:
def clean_text_by_word(text):
    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]
    
    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)
        
    return " ".join(m_words)

In [26]:
for i in range(len(df)):
    texto = clean_text_by_word(df.loc[i, 'texto'])
    df.loc[i, 'texto'] = texto

In [27]:
for i in range(len(data)):
    for j in range(len(df)):
        if data['id_ato'][i] == df['id_ato'][j]:
            entitie = re.sub('xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?', '', clean_text_by_word(data['texto'][i]).replace("\r", ""))
            df.loc[j, data['tipo_ent'][i]] = entitie

In [28]:
df['texto'][0]

'EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO No 002/2018, NOS TERMOS DO PADRAO No 13/2002 SUPLEMENTACAO SUPLEMENTACAO Processo:0131-000016/2018. Partes: DF/RA II, CNPJ no 33.524.869/0001-94 e a COMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL CAESB, CAESB, CNPJ no 00.082.024/0001-37. Objeto: O presente Termo Aditivo objetiva a alteracao contratual com vistas a SUPLEMENTACAO de recursos no valor de R$ 27.900,00 (vinte e sete mil e novecentos reais), ao Contrato no 002/2018, celebrado em 30/01/2018, Extrato do Contrato publicado no DODF n 34 de 20 de fevereiro de 2018, pagina 39, nos termos do inciso I, alinea "b", art. 65 da Lei no 8.666/93, passando o Contrato a ter o valor total de R$ 251.015,81 (duzentos e cinquenta e um mil quinze reais e oitenta e um centavos). Da Dotacao Orcamentaria: a despesa decorrente da presente alteracao correra a conta da seguinte Dotacao Orcamentaria: I Unidade Unidade Orcamentaria: 09104 Administracao Administracao Regional do Gama, II Programa Progr

### IOB

In [ ]:
from nltk.tokenize import word_tokenize

In [31]:
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == tokenizer(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in tokenizer(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(tokenizer(row[entity])) and \
                    token == tokenizer(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(tokenizer(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)


In [32]:
iob = IOBifyer(column='texto')
r = iob.transform(df)

In [33]:
r

0      [O, O, B-numero_termo_aditivo, I-numero_termo_...
1      [O, O, B-numero_termo_aditivo, I-numero_termo_...
2      [O, O, O, B-modalidade_licitacao, I-modalidade...
3      [B-numero_termo_aditivo, I-numero_termo_aditiv...
4      [O, B-numero_termo_aditivo, I-numero_termo_adi...
                             ...                        
721    [O, O, O, B-modalidade_licitacao, I-modalidade...
722    [O, O, O, O, O, B-modalidade_licitacao, I-moda...
723    [O, O, O, B-modalidade_licitacao, I-modalidade...
724    [O, B-processo_gdf, I-processo_gdf, I-processo...
725    [O, O, O, O, O, O, B-numero_contrato, I-numero...
Length: 726, dtype: object

In [34]:
df.shape, r.shape

((726, 44), (726,))

In [35]:
df["IOB"] = np.nan

In [36]:
for i in range(len(df)):
    df.loc[i, "IOB"] = ' '.join(r[i])

In [37]:
df['texto'][0]

'EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO No 002/2018, NOS TERMOS DO PADRAO No 13/2002 SUPLEMENTACAO SUPLEMENTACAO Processo:0131-000016/2018. Partes: DF/RA II, CNPJ no 33.524.869/0001-94 e a COMPANHIA DE SANEAMENTO AMBIENTAL DO DISTRITO FEDERAL CAESB, CAESB, CNPJ no 00.082.024/0001-37. Objeto: O presente Termo Aditivo objetiva a alteracao contratual com vistas a SUPLEMENTACAO de recursos no valor de R$ 27.900,00 (vinte e sete mil e novecentos reais), ao Contrato no 002/2018, celebrado em 30/01/2018, Extrato do Contrato publicado no DODF n 34 de 20 de fevereiro de 2018, pagina 39, nos termos do inciso I, alinea "b", art. 65 da Lei no 8.666/93, passando o Contrato a ter o valor total de R$ 251.015,81 (duzentos e cinquenta e um mil quinze reais e oitenta e um centavos). Da Dotacao Orcamentaria: a despesa decorrente da presente alteracao correra a conta da seguinte Dotacao Orcamentaria: I Unidade Unidade Orcamentaria: 09104 Administracao Administracao Regional do Gama, II Programa Progr

In [38]:
len(DEFAULT_TOKENIZER(df['texto'][0])), len(df['IOB'][0].split())

(298, 298)

In [39]:
df['IOB'][0]

'O O B-numero_termo_aditivo I-numero_termo_aditivo I-numero_termo_aditivo O O O B-numero_contrato I-numero_contrato O O O O O O O O O O B-processo_gdf I-processo_gdf I-processo_gdf O B-orgao_contratante I-orgao_contratante I-orgao_contratante O O O O O O O O O O O O O O O O O O O O O O O O O O B-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_contratual I-objeto_aditamento_con

In [41]:
# df.to_parquet('../acts_parquet/iob_contratos_idato.parquet')

In [42]:
df = pd.read_parquet('../acts_parquet/iob_contratos_idato.parquet')
df

,id_ato,id_dodf,tipo_rel,tipo_ent,texto,numero_termo_aditivo,numero_contrato,orgao_contratante,objeto_aditamento_contratual,processo_gdf,...,cnpj_entidade_convenente,vigencia_convenio,valor_convenio,numero_convenio,objeto_convenio,data_assinatura_convenio,identificacao_ocorrencia,cnpj_orgao_contratante,cnpj_orgao_concedente,IOB
0,22_20.12.2019.1-R5,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO No...,QUARTO TERMO ADITIVO,002/2018,DF/RA II,O presente Termo Aditivo objetiva a alteracao ...,0131-000016/2018,...,None,None,None,None,None,None,None,None,None,O O B-numero_termo_aditivo I-numero_termo_adit...
1,22_20.12.2019.1-R6,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,EXTRATO DO QUARTO TERMO ADITIVO AO CONTRATO No...,QUARTO TERMO ADITIVO,002/2018,DF/RA II,O presente Termo Aditivo objetiva a alteracao ...,0131-000016/2018,...,None,None,None,None,None,None,None,None,None,O O B-numero_termo_aditivo I-numero_termo_adit...
2,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO No 001/20...,None,None,None,None,00040-00017799/2019-42,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
3,22_20.12.2019.1-R7,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,III TERMO ADITIVO AO CONTRATO BRB No 064/2017 ...,III TERMO ADITIVO,064/2017,BRB BANCO BANCO DE BRASILIA S/A.,acrescer qualitativamente o contrato para incl...,041.001.009/2015,...,None,None,None,None,None,None,None,None,None,B-numero_termo_aditivo I-numero_termo_aditivo ...
4,22_20.12.2019.1-R8,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,Especie: Quarto Termo Aditivo ao Contrato no 1...,Quarto Termo Aditivo,114/2015,SECRETARIA DE ESTADO DE SAUDE DO DISTRITO FEDERAL,"Prorrogar o prazo de vigencia do Contrato, por...",0060-006232/2015,...,None,None,None,None,None,None,None,None,None,O B-numero_termo_aditivo I-numero_termo_aditiv...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,8_13.8.2019-R14,8_13.8.2019,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO No 27/201...,None,None,None,None,0052.002.486/2017,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
722,8_13.8.2019-R6,8_13.8.2019,REL_ANUL_REVOG_LICITACAO,AVISO_ANUL_REV_LICITACAO,AVISO DE REVOGACAO DE LICITACAO PREGAO ELETRON...,None,None,None,None,00400-00023725/2019-17,...,None,None,None,None,None,None,revoga,None,None,O O O O O B-modalidade_licitacao I-modalidade_...
723,8_13.8.2019-R13,8_13.8.2019,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO PE No 3/2...,None,None,None,None,00400-00023725/2019-17,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
724,8_13.8.2019-R12,8_13.8.2019,REL_ADITAMENTO_CONTRATO,EXTRATO_ADITAMENTO_CONTRATUAL,PROCESSO: 0112-001373/2016. ESPECIE: TERCEIRO ...,TERCEIRO,566/2019,COMPANHIA URBANIZADORA DA NOVA CAPITAL DO BRAS...,"Prorrogacao do prazo de vigencia, bem como o r...",0112-001373/2016,...,None,None,None,None,None,None,None,None,None,O B-processo_gdf I-processo_gdf I-processo_gdf...


In [48]:
df.loc[(df['tipo_ent'] == 'AVISO_LICITACAO')]

,id_ato,id_dodf,tipo_rel,tipo_ent,texto,numero_termo_aditivo,numero_contrato,orgao_contratante,objeto_aditamento_contratual,processo_gdf,...,cnpj_entidade_convenente,vigencia_convenio,valor_convenio,numero_convenio,objeto_convenio,data_assinatura_convenio,identificacao_ocorrencia,cnpj_orgao_contratante,cnpj_orgao_concedente,IOB
2,22_20.12.2019.1-R20,22_20.12.2019.1,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO No 001/20...,None,None,None,None,00040-00017799/2019-42,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
7,22_20.12.2019.1-R19,22_20.12.2019.1,REL_AVISO_LICITACAO,AVISO_LICITACAO,PREGAO ELETRONICO No 03/2019 UASG UASG 925046 ...,None,None,None,None,00097-00003659/2018-32,...,None,None,None,None,None,None,None,None,None,B-modalidade_licitacao I-modalidade_licitacao ...
10,22_20.12.2019.1-R22,22_20.12.2019.1,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE ABERTURA DE LICITACAO SRP SRP PREGAO ...,None,None,None,None,00050-00034379/2019-93,...,None,None,None,None,None,None,None,None,None,O O O O O O O B-modalidade_licitacao I-modalid...
14,22_20.12.2019.1-R21,22_20.12.2019.1,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISOS DE LICITACAO PREGAO ELETRONICO PE No 30...,None,None,None,None,092.006186/2019,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
18,12_22.11.2018-R21,12_22.11.2018,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISOS DE LICITACOES PREGAO ELETRONICO No 116/...,None,None,None,None,00410-00024534/2017-65,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,9_10.9.2018-R22,9_10.9.2018,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE ABERTURA DE LICITACAO PREGAO ELETRONI...,None,None,None,None,050.001.370/2017,...,None,None,None,None,None,None,None,None,None,O O O O O B-modalidade_licitacao I-modalidade_...
705,9_10.9.2018-R23,9_10.9.2018,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO Pregao Eletronico no 041/20...,None,None,None,None,00112-00008000/2018-55,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
706,9_10.9.2018-R24,9_10.9.2018,REL_AVISO_LICITACAO,AVISO_LICITACAO,"AVISO DE LICITACAO COTA COTA RESERVADA, REQUIS...",None,None,None,None,092.004741/2018,...,None,None,None,None,None,None,None,None,None,O O O O O O O B-modalidade_licitacao I-modalid...
721,8_13.8.2019-R14,8_13.8.2019,REL_AVISO_LICITACAO,AVISO_LICITACAO,AVISO DE LICITACAO PREGAO ELETRONICO No 27/201...,None,None,None,None,0052.002.486/2017,...,None,None,None,None,None,None,None,None,None,O O O B-modalidade_licitacao I-modalidade_lici...
